In [1]:
import numpy as np
import torch

import lightdde as dde

In [2]:
Re = 20
nu = 1 / Re
l = 1 / (2 * nu) - np.sqrt(1 / (4 * nu ** 2) + 4 * np.pi ** 2)

In [14]:
def pde(x, u):
    u_vel, v_vel, p = u[:, 0:1], u[:, 1:2], u[:, 2:]
    u_vel_x = dde.grad.jacobian(u, x, i=0, j=0)
    u_vel_y = dde.grad.jacobian(u, x, i=0, j=1)
    u_vel_xx = dde.grad.hessian(u, x, component=0, i=0, j=0)
    u_vel_yy = dde.grad.hessian(u, x, component=0, i=1, j=1)

    v_vel_x = dde.grad.jacobian(u, x, i=1, j=0)
    v_vel_y = dde.grad.jacobian(u, x, i=1, j=1)
    v_vel_xx = dde.grad.hessian(u, x, component=1, i=0, j=0)
    v_vel_yy = dde.grad.hessian(u, x, component=1, i=1, j=1)

    p_x = dde.grad.jacobian(u, x, i=2, j=0)
    p_y = dde.grad.jacobian(u, x, i=2, j=1)

    momentum_x = (
        u_vel * u_vel_x + v_vel * u_vel_y + p_x - 1 / Re * (u_vel_xx + u_vel_yy)
    )
    momentum_y = (
        u_vel * v_vel_x + v_vel * v_vel_y + p_y - 1 / Re * (v_vel_xx + v_vel_yy)
    )
    continuity = u_vel_x + v_vel_y

    return [momentum_x, momentum_y, continuity]

In [15]:
class FNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.activation = torch.nn.Tanh()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(2, 50),
            self.activation,
            torch.nn.Linear(50, 50),
            self.activation,
            torch.nn.Linear(50, 50),
            self.activation,
            torch.nn.Linear(50, 50),
            self.activation,
            torch.nn.Linear(50, 3),
        )

    def forward(self, x):
        x = x.view(-1, 2)
        return self.net(x)

In [16]:
net = FNN()

In [17]:
x = np.random.randn(100, 2).astype(np.float32)
x = torch.from_numpy(x).requires_grad_()

In [18]:
y = net(x)

In [19]:
m_x, m_y, cont = pde(x, y)

In [20]:
m_x

tensor([[0.0391],
        [0.0429],
        [0.0371],
        [0.0414],
        [0.0289],
        [0.0142],
        [0.0229],
        [0.0343],
        [0.0077],
        [0.0192],
        [0.0239],
        [0.0352],
        [0.0423],
        [0.0165],
        [0.0343],
        [0.0407],
        [0.0232],
        [0.0409],
        [0.0061],
        [0.0361],
        [0.0377],
        [0.0391],
        [0.0071],
        [0.0363],
        [0.0297],
        [0.0279],
        [0.0180],
        [0.0244],
        [0.0360],
        [0.0115],
        [0.0102],
        [0.0408],
        [0.0080],
        [0.0210],
        [0.0405],
        [0.0212],
        [0.0386],
        [0.0194],
        [0.0285],
        [0.0356],
        [0.0387],
        [0.0343],
        [0.0261],
        [0.0362],
        [0.0412],
        [0.0177],
        [0.0119],
        [0.0358],
        [0.0180],
        [0.0054],
        [0.0307],
        [0.0154],
        [0.0085],
        [0.0009],
        [0.0135],
        [0